In [12]:
# Open the 'input.txt' file in read mode with UTF-8 encoding
with open('input.txt', 'r', encoding='utf-8') as f:
    # Read the contents of the file and store it in the 'text' variable
    text = f.read()

# Print the length of the dataset in characters
print("length of dataset in characters: ", len(text))

# Print the first 1000 characters of the 'text' variable
print(text[:1000])

length of dataset in characters:  1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hung

In [13]:
# Tokenizer Detokenizer

# ticktoken is a simple encoder and decoder but for our applications
# we will use a simple encoding
chars = sorted(list(set(text)))

stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])


In [14]:
#%pip install torch
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [15]:
# Splitting the data into train and validation
n = int(0.9 * len(data))
train_data, val_data = data[:n], data[n:]

block_size = 8
 

In [17]:
# Each train sample contains 8 sub train samples
# This is demonstrated below
 
x=train_data[:block_size]
y=train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t]
    target = y[t]
    print(f'when input is {context} target is {target}')

when input is tensor([], dtype=torch.int64) target is 47
when input is tensor([18]) target is 56
when input is tensor([18, 47]) target is 57
when input is tensor([18, 47, 56]) target is 58
when input is tensor([18, 47, 56, 57]) target is 1
when input is tensor([18, 47, 56, 57, 58]) target is 15
when input is tensor([18, 47, 56, 57, 58,  1]) target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15]) target is 58


In [18]:
# Loading the data in batches

torch.manual_seed(1337)
batch_size = 4
block_size = 8 # Maximal context length for predictions

def get_batch(split):

    data = train_data if split == 'train' else val_data

    # Generate random starting indices for each batch
    starts = torch.randint(0, data.size(0) - block_size, (batch_size,))
    
    # Construct batches
    x = torch.stack([data[i:i+block_size] for i in starts])
    # The target is the original sequence, shifted by one
    y = torch.stack([data[i+1:i+block_size+1] for i in starts])

    return x, y

x, y = get_batch('train')
print(f'{x.shape=}, {y.shape=}')
print('-----')

x.shape=torch.Size([4, 8]), y.shape=torch.Size([4, 8])
-----


In [ ]:
# Model Creation
import torch
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, target):
        logits = self.token_embedding_table(idx) # (B T C)
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        target = target.view(-1)
        loss = F.cross_entropy(logits, target)

        return logits, loss

m = BigramLanguageModel(vocab_size=len(chars))
out, loss = m(x, y)
print(out.shape)
print(loss.shape)